<a href="https://colab.research.google.com/github/heywassup7/Git-Tutorial/blob/master/LSTM_%EB%AA%A8%EB%8D%B8_%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from sklearn.model_selection import train_test_split

In [32]:
# CSV 파일 가져오기

# 코랩 사용 시 아래 두 줄 주석 해제
# from google.colab import files
# uploaded = files.upload()

csv_df = pd.read_csv('dataset.csv')
csv_df

,Unnamed: 0,Date,환율,금리,y,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR,종가
0,0,2013-01-01,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
1,1,2013-01-02,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
2,2,2013-01-03,1062.599976,2.75,30860.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
3,3,2013-01-04,1061.900024,2.75,30500.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
4,4,2013-01-07,1033.099976,2.75,30400.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,3647,2022-12-18,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3648,3648,2022-12-24,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3649,3649,2022-12-25,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3650,3650,2022-12-31,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0


In [45]:
# 날짜 인덱스가 없음 일단 임시로
csv_df = csv_df.rename(columns={'Unnamed: 0':"날짜"})
csv_df

,날짜,Date,환율,금리,y,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR,종가
0,0,2013-01-01,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
1,1,2013-01-02,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
2,2,2013-01-03,1062.599976,2.75,30860.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
3,3,2013-01-04,1061.900024,2.75,30500.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
4,4,2013-01-07,1033.099976,2.75,30400.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,3647,2022-12-18,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3648,3648,2022-12-24,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3649,3649,2022-12-25,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3650,3650,2022-12-31,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0


In [52]:
#날짜 빼고 정규화

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scale_cols = ['ROE','영업이익률','부채비율','EPS','순이익률','BPS','y','PER','PBR']

#정규화 수행
scaled_df = scaler.fit_transform(csv_df[scale_cols]) #정규화된 데이터는 넘파이 형태
scaled_df = pd.DataFrame(scaled_df, columns = scale_cols) #Pandas DataFrame 형태로 변경

#테스트
scaled_df

,ROE,영업이익률,부채비율,EPS,순이익률,BPS,y,PER,PBR
0,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.146138,0.000000,0.003474
1,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.146138,0.000000,0.003474
2,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.136664,0.000000,0.003474
3,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.131496,0.000000,0.003474
4,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.130060,0.000000,0.003474
...,...,...,...,...,...,...,...,...,...
3647,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.318117,0.321004,0.528596
3648,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.318117,0.321004,0.528596
3649,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.318117,0.321004,0.528596
3650,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.318117,0.321004,0.528596


In [53]:
scaled_csv_df = pd.concat([csv_df['날짜'], scaled_df], axis=1)
scaled_csv_df = scaled_csv_df.set_index('날짜')
scaled_csv_df

,ROE,영업이익률,부채비율,EPS,순이익률,BPS,y,PER,PBR
날짜,,,,,,,,,
0,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.146138,0.000000,0.003474
1,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.146138,0.000000,0.003474
2,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.136664,0.000000,0.003474
3,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.131496,0.000000,0.003474
4,1.000000,0.333160,1.0,1.000000,0.433271,0.816624,0.130060,0.000000,0.003474
...,...,...,...,...,...,...,...,...,...
3647,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.318117,0.321004,0.528596
3648,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.318117,0.321004,0.528596
3649,0.666542,0.190005,0.0,0.025159,1.000000,0.021155,0.318117,0.321004,0.528596


In [54]:
#입력데이터, 정답데이터 정의
feature_cols = ['ROE','영업이익률','부채비율','EPS','순이익률','BPS','PER','PBR'] 
label_cols = ['y'] 

#입력, 정답데이터 프레임
feature_df = scaled_csv_df[feature_cols]
label_df = scaled_csv_df[label_cols]

print(feature_df)
print(label_df)

feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

           ROE     영업이익률  부채비율       EPS      순이익률       BPS       PER   
날짜                                                                       
0     1.000000  0.333160   1.0  1.000000  0.433271  0.816624  0.000000  \
1     1.000000  0.333160   1.0  1.000000  0.433271  0.816624  0.000000   
2     1.000000  0.333160   1.0  1.000000  0.433271  0.816624  0.000000   
3     1.000000  0.333160   1.0  1.000000  0.433271  0.816624  0.000000   
4     1.000000  0.333160   1.0  1.000000  0.433271  0.816624  0.000000   
...        ...       ...   ...       ...       ...       ...       ...   
3647  0.666542  0.190005   0.0  0.025159  1.000000  0.021155  0.321004   
3648  0.666542  0.190005   0.0  0.025159  1.000000  0.021155  0.321004   
3649  0.666542  0.190005   0.0  0.025159  1.000000  0.021155  0.321004   
3650  0.666542  0.190005   0.0  0.025159  1.000000  0.021155  0.321004   
3651  0.666542  0.190005   0.0  0.025159  1.000000  0.021155  0.321004   

           PBR  
날짜              
0  

In [55]:
window_size = 2   #40개의 입력데이터를 이용해 바로 다음 값에 오는 Close 값을 예측

def make_sequence_dataset(feature, label, window_size):

  feature_list = []   #생성될 feature list
  label_list = []     #생성될 label list

  for i in range(len(feature)-window_size):

    feature_list.append(feature[i:i+window_size])
    label_list.append(label[i+window_size])

  return np.array(feature_list), np.array(label_list)


X, y = make_sequence_dataset(feature_np, label_np, window_size)

print(X.shape, y.shape)

(3650, 2, 8) (3650, 1)


In [56]:
split = int(len(feature_df)*0.7) #테스트 데이터로 분리

X_train = X[0:split]
y_train = y[0:split]

X_test = X[split:]
y_test = y[split:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2556, 2, 8) (2556, 1)
(1094, 2, 8) (1094, 1)


In [57]:
#LSTM 모델 구축

model = Sequential()

#LSTM layer
model.add(LSTM(128,
               activation = 'sigmoid',
               input_shape = X_train[0].shape))   #input_shape = (2,8)
#출력층
model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 128)               70144     
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 70,273
Trainable params: 70,273
Non-trainable params: 0
_________________________________________________________________


In [58]:
# 모델 컴파일
model.compile(loss='mse', optimizer='adam', metrics=['mae'])  # 옵티마이저와 손실 함수, 평가지표 설정

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32)

# 주가 예측
predictions = model.predict(X_test)

Epoch 1/50
80/80 [==============================] - 1s 1ms/step - loss: 0.0618 - mae: 0.1918
Epoch 2/50
80/80 [==============================] - 0s 1ms/step - loss: 0.0291 - mae: 0.1256
Epoch 3/50
80/80 [==============================] - 0s 1ms/step - loss: 0.0277 - mae: 0.1250
Epoch 4/50
80/80 [==============================] - 0s 2ms/step - loss: 0.0264 - mae: 0.1212
Epoch 5/50
80/80 [==============================] - 0s 1ms/step - loss: 0.0253 - mae: 0.1187
Epoch 6/50
80/80 [==============================] - 0s 1ms/step - loss: 0.0240 - mae: 0.1171
Epoch 7/50
80/80 [==============================] - 0s 1ms/step - loss: 0.0235 - mae: 0.1173
Epoch 8/50
80/80 [==============================] - 0s 1ms/step - loss: 0.0240 - mae: 0.1210
Epoch 9/50
80/80 [==============================] - 0s 1ms/step - loss: 0.0214 - mae: 0.1119
Epoch 10/50
80/80 [==============================] - 0s 1ms/step - loss: 0.0211 - mae: 0.1124
Epoch 11/50
80/80 [==============================] - 0s 1ms/step - lo

In [60]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# MAE, RMSE, R-Squared 계산
mae = mean_absolute_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
#r2 = r2_score(y_test, predictions)

# 결과 출력
print("MAE: {:.5f}".format(mae))
print("RMSE: {:.5f}".format(rmse))
#print("R-Squared: {:.5f}".format(r2))  

##r2_score를 사용하기에는 샘플 수도 적고, 시계열 데이터 특성상 안 맞을 수도 있을 것 같음.
##지금은 테스트 샘플이 2개 이하라서 nan 값이 나오고, 3개로 늘려봤을 때도 -5000이하의 값이 나온다...

MAE: 0.54551
RMSE: 0.59055


In [61]:
#종가 데이터만 모두 불러오고, 나머지 데이터는 그 해 년도를 그대로 적용하여 사용 -> 할 수 있을 것 같음

In [78]:
print(predictions)

# predictions = scaler.inverse_transform(predictions)
#scaler.inverse_transform(predictions)

[[ 0.5691288 ]
 [ 0.5691288 ]
 [ 0.5691288 ]
 ...
 [-0.36159256]
 [-0.36159256]
 [-0.36159256]]


In [76]:
predictions = scaler.inverse_transform(predictions)

ValueError: non-broadcastable output operand with shape (1094,1) doesn't match the broadcast shape (1094,9)